# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: I'm going to be using LoRA since I wasn't sure what else to use.
* Model: I'm using distilbert-base-uncased as my model.
* Evaluation approach: I'm using the evaluate method with Hugging Face Trainer.
* Fine-tuning dataset: I'm going to be using the derek-thomas ScienceQA dataset.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [5]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [6]:
dataset = load_dataset("stanfordnlp/imdb", split="train[:1%]").train_test_split(
test_size=0.2, shuffle=True, seed=23
)
def preprocess_function(examples):
    """Preprocess the dataset by returning tokenized examples."""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
tokenized_ds = dataset.map(preprocess_function, batched=True)
training_args = TrainingArguments(
    output_dir=".",
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer,
    padding="max_length"),
    compute_metrics=compute_metrics,
)
eval_results = trainer.evaluate()
print("Base Model Evaluation Results:", eval_results)
print(tokenized_ds["train"][0]["input_ids"])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Base Model Evaluation Results: {'eval_loss': 0.7533522248268127, 'eval_accuracy': 0.02, 'eval_runtime': 83.0461, 'eval_samples_per_second': 0.602, 'eval_steps_per_second': 0.084}
[101, 1037, 2944, 2315, 6874, 2003, 2551, 1999, 2148, 2637, 2043, 2016, 2003, 11364, 2013, 2014, 3309, 2282, 2011, 1037, 6080, 1998, 2579, 2046, 1996, 8894, 1012, 2027, 5157, 1037, 4121, 16540, 2005, 2014, 2713, 1012, 2848, 2003, 5086, 2000, 2131, 2014, 2188, 3647, 1998, 2045, 2003, 1037, 6781, 1999, 2009, 2005, 2032, 2065, 2002, 2064, 3288, 2067, 1996, 2769, 2004, 2092, 2004, 1996, 2611, 1012, 2848, 2003, 2579, 2000, 1996, 8894, 2011, 7739, 2007, 2767, 2990, 1012, 2027, 3046, 2000, 2507, 1996, 22590, 7347, 8275, 2769, 1999, 2709, 2005, 6874, 2021, 1996, 2933, 3632, 27762, 3308, 1998, 2027, 2031, 2000, 15358, 2041, 1999, 1996, 7739, 1012, 1996, 7739, 2003, 2915, 1998, 2027, 2036, 2031, 2000, 15358, 2013, 2008, 1006, 2025, 2183, 2092, 2061, 2521, 1007, 1012, 102]


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [7]:
config = LoraConfig()
lora_config = LoraConfig(
task_type=TaskType.SEQ_CLS,
r=8,
lora_alpha=16,
lora_dropout=0.1,
target_modules=["q_lin", "k_lin", "v_lin"]
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

dataset = load_dataset("stanfordnlp/imdb", split="train[:10]").train_test_split(
test_size=0.2, shuffle=True, seed=23
)

trainable params: 1,405,444 || all params: 67,768,324 || trainable%: 2.073895172617815


In [8]:
small_train_dataset = dataset["train"].select(range(int(len(dataset["train"]) * 0.1)))
small_eval_dataset = dataset["test"].select(range(int(len(dataset["train"]) * 0.1)))
print(small_train_dataset.column_names)

['text', 'label']


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"].shard(num_shards=2, index=0)
test_dataset = tokenized_datasets["test"].shard(num_shards=2, index=1)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    remove_unused_columns=False
)
trainer = Trainer(
    model=lora_model,
    train_dataset=train_dataset,#.rename_column('label', 'labels'),
    eval_dataset=eval_dataset,#.rename_column('label', 'labels'),
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")
)
trainer.train()
lora_model.save_pretrained("gpt-lora")
tokenizer.save_pretrained("gpt-lora")

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.688657


('gpt-lora/tokenizer_config.json',
 'gpt-lora/special_tokens_map.json',
 'gpt-lora/vocab.txt',
 'gpt-lora/added_tokens.json',
 'gpt-lora/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [3]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt-lora")
tokenizer.pad_token = tokenizer.eos_token
model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")
dataset = load_dataset("stanfordnlp/imdb", split="train[:10]").train_test_split(
test_size=0.2, shuffle=True, seed=23
)
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) 
tokenized_datasets = dataset.map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=512), batched=True, remove_columns=["text"])
eval_dataset = tokenized_datasets["test"]
def evaluate_model(model, eval_dataset, tokenizer):
    total_loss = 0
    num_batches = 0   
    for example in eval_dataset:
        inputs = {k: torch.tensor(v).unsqueeze(0) for k, v in example.items()}
        inputs['labels'] = inputs.pop('label')
        
        print("Processing example, inputs:", inputs)
        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs.loss
            total_loss += loss.item()
            num_batches += 1
    average_loss = total_loss / num_batches if num_batches != 0 else 0
    return average_loss

peft_model_loss = evaluate_model(model, eval_dataset, tokenizer)
print(f"PEFT Model Loss: {peft_model_loss}")

original_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
original_model_loss = evaluate_model(original_model, eval_dataset, tokenizer)
print(f"Original Model Loss: {original_model_loss}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing example, inputs: {'input_ids': tensor([[  101,  2023,  2143,  2001,  2763,  4427,  2011,  2643,  4232,  1005,
          1055, 16137, 10841,  4115,  1010, 10768, 25300,  2078,  1998,  1045,
          9075,  2017,  2000,  2156,  2008,  2143,  2612,  1012,  1026,  7987,
          1013,  1028,  1026,  7987,  1013,  1028,  1996,  2143,  2038,  2048,
          2844,  3787,  1998,  2216,  2024,  1010,  1006,  1015,  1007,  1996,
         12689,  3772,  1006,  1016,  1007,  1996,  8052,  1010,  6151,  6810,
          2099,  7178,  2135,  2204,  1010,  6302,  1012,  4237,  2013,  2008,
          1010,  2054,  9326,  2033,  2087,  2003,  1996, 10866,  5460,  1997,
          9033, 21202,  7971,  1012, 14229,  6396,  2386,  2038,  2000,  2022,
          2087, 15703,  3883,  1999,  1996,  2088,  1012,  2016,  4490,  2061,
          5236,  1998,  2007,  2035,  1996, 16371, 25469,  1999,  2023,  2143,
          1010,  1012,  1012,  1012,  2009,  1005,  1055, 14477,  4779, 26884,
          

PEFT Model Loss: 0.5642923712730408


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing example, inputs: {'input_ids': tensor([[  101,  2023,  2143,  2001,  2763,  4427,  2011,  2643,  4232,  1005,
          1055, 16137, 10841,  4115,  1010, 10768, 25300,  2078,  1998,  1045,
          9075,  2017,  2000,  2156,  2008,  2143,  2612,  1012,  1026,  7987,
          1013,  1028,  1026,  7987,  1013,  1028,  1996,  2143,  2038,  2048,
          2844,  3787,  1998,  2216,  2024,  1010,  1006,  1015,  1007,  1996,
         12689,  3772,  1006,  1016,  1007,  1996,  8052,  1010,  6151,  6810,
          2099,  7178,  2135,  2204,  1010,  6302,  1012,  4237,  2013,  2008,
          1010,  2054,  9326,  2033,  2087,  2003,  1996, 10866,  5460,  1997,
          9033, 21202,  7971,  1012, 14229,  6396,  2386,  2038,  2000,  2022,
          2087, 15703,  3883,  1999,  1996,  2088,  1012,  2016,  4490,  2061,
          5236,  1998,  2007,  2035,  1996, 16371, 25469,  1999,  2023,  2143,
          1010,  1012,  1012,  1012,  2009,  1005,  1055, 14477,  4779, 26884,
          

Original Model Loss: 0.6710483133792877
